In [1]:
import nltk
from nltk.corpus import cmudict

entries = cmudict.entries()

In [2]:
mylist = []
for w in range(len(entries)):
    for x in range(len(entries[w][1])):
        if entries[w][1][x] == 'SH':
            mylist.append(entries[w])
            break

In [3]:
words = []
phone = []
for w in mylist:
    words.append(w[0])
    phone.append(w[1])

In [4]:
vowels1 = ['AA1', 'AE1', 'AH1', 'AO1', 'AW1', 'AX1', 'AXR1', 'AY1', 'EH1', 'ER1', 'EY1',
         'IH1', 'IY1', 'IX1', 'OW1', 'OY1', 'UH1', 'UW1', 'UX1']

vowels0 = []
for w in vowels1:
    for x in range(len(w)):
        if w[x] == '1':
            vowels0.append(w[:x] + '0')
vowels2 = []
for w in vowels1:
    for x in range(len(w)):
        if w[x] == '1':
            vowels2.append(w[:x] + '2')

vowels = vowels0 + vowels1 + vowels2

In [5]:
intervocalic = []
preceding = []
following = []
for w in phone:
    for x in range(len(w)):
        if w[x] == 'SH':
            if x == 0:
                intervocalic.append(0)
                preceding.append('0')
                if (x+1 in range(len(w))) and (w[x+1] in vowels):
                    following.append(w[x+1])
                    break
                else:
                    following.append('0')
                    break
            elif x+1 in range(len(w)):
                if ((w[x-1] in vowels) and (w[x+1] in vowels)):
                    intervocalic.append(1)
                    preceding.append(w[x-1])
                    following.append(w[x+1])
                    break
                elif ((w[x-1] in vowels) and (w[x+1] not in vowels)):
                    intervocalic.append(0)
                    preceding.append(w[x-1])
                    following.append('0')
                    break
                elif ((w[x-1] not in vowels) and (w[x+1] in vowels)):
                    intervocalic.append(0)
                    preceding.append('0')
                    following.append(w[x+1])
                    break
                else:
                    intervocalic.append(0)
                    preceding.append('0')
                    following.append('0')
                    break
            elif x+1 not in range(len(w)):
                if w[x-1] in vowels:
                    intervocalic.append(0)
                    preceding.append(w[x-1])
                    following.append('0')
                    break
                else:
                    intervocalic.append(0)
                    preceding.append('0') #is this bug?
                    following.append('0')
                    break

In [6]:
monodict = {}
count = 0
for w in phone:
    for x in range(len(w)):
        if w[x] in vowels:
            count += 1
    if count == 1:
        monodict[str(w)] = 1
        count = 0
    else:
        monodict[str(w)] = 0
        count = 0

In [7]:
#when followed by a vowel, esh is the onset of the syll, when preceded by a vowel and not followed by a vowel,
#esh is the coda of the previous syll.
stress = []
stressdeg = []
for w in phone:
    if monodict[str(w)] == 1:
        stress.append(1)
        stressdeg.append(1)
    else:    
        for x in range(len(w)):
            if w[x] == 'SH':
                #abash, sh is the final phone, thus the coda, thus look at preceding vowel
                if (x+1) not in range(len(w)):
                    if w[x-1] in vowels1:
                        stress.append(1)
                        stressdeg.append(1)
                        break
                    elif w[x-1] in vowels2:
                        stress.append(1)
                        stressdeg.append(2)
                        break
                    else:
                        stress.append(0)
                        stressdeg.append(0)
                        break
                #sh is not the final phone
                elif (x+1) in range(len(w)):
                    #if followed by a vowel, sh is the onset
                    if w[x+1] in vowels1:
                        stress.append(1)
                        stressdeg.append(1)
                        break
                    elif w[x+1] in vowels2:
                        stress.append(1)
                        stressdeg.append(2)
                        break
                    elif w[x+1] in vowels0:
                        stress.append(0)
                        stressdeg.append(0)
                        break
                    #sh is not final but not followed by a vowel, look at preceding phone, sh might be coda or cluster
                    else:
                        if w[x-1] in vowels1:
                            stress.append(1)
                            stressdeg.append(1)
                            break
                        elif w[x-1] in vowels2:
                            stress.append(1)
                            stressdeg.append(2)
                            break
                        else:
                            stress.append(0)
                            stressdeg.append(0)
                            break

In [8]:
phone_clean = []
for w in phone:
    entry = ""
    for x in range(len(w)):
        entry = entry + w[x] + ' '
    phone_clean.append(entry)

In [9]:
import pandas
colnames = ["Word","FREQcount","CDcount","FREQlow","Cdlow","SUBTLWF","Lg10WF","SUBTLCD","Lg10CD","Dom_PoS_SUBTLEX","Freq_dom_PoS_SUBTLEX","Percentage_dom_PoS","All_PoS_SUBTLEX","All_freqs_SUBTLEX","Zipf"]
data = pandas.read_csv("Freqplus.csv", names = colnames, low_memory=False)

In [10]:
Word = data.Word.tolist()
SUBTLWF = data.SUBTLWF.tolist()
Lg10WF = data.Lg10WF.tolist()
Zipf = data.Zipf.tolist()

In [11]:
mydict = {Word[i]: (SUBTLWF[i], Lg10WF[i], Zipf[i]) for i in range(len(Word))}

In [12]:
my_SUBTLWF = []
my_Lg10WF = []
my_Zipf = []
for w in words:
    try:
        my_SUBTLWF.append(mydict[w][0])
        my_Lg10WF.append(mydict[w][1])
        my_Zipf.append(mydict[w][2])
    except KeyError:
        my_SUBTLWF.append('0')
        my_Lg10WF.append('0')
        my_Zipf.append('0')

In [13]:
import csv
from itertools import zip_longest

d = [words, phone_clean, intervocalic, preceding, following, stress, stressdeg, my_SUBTLWF, my_Lg10WF, my_Zipf]
export_data = zip_longest(*d, fillvalue = '')

with open('numbers.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("words", "phone", "intervocalic", "preceding", "following", "stress", "stressdeg", "SUBTLWF", "Lg10WF", "Zipf"))
      wr.writerows(export_data)
        
myfile.close()